拥有多个相关的模型是很常见的。

对用户模型来说尤其如此，因为：

输入模型需要拥有密码属性。
输出模型不应该包含密码。
数据库模型很可能需要保存密码的

下面是应该如何根据它们的密码字段以及使用位置去定义模型的大概思路：哈希值。

In [2]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel, EmailStr

app = FastAPI()


class UserIn(BaseModel):
    username: str
    password: str
    email: EmailStr
    full_name: str | None = None


class UserOut(BaseModel):
    username: str
    email: EmailStr
    full_name: str | None = None


class UserInDB(BaseModel):
    username: str
    hashed_password: str
    email: EmailStr
    full_name: str | None = None


def fake_password_hasher(raw_password: str):
    return "supersecret" + raw_password


def fake_save_user(user_in: UserIn):
    hashed_password = fake_password_hasher(user_in.password)
    user_in_db = UserInDB(**user_in.model_dump(), hashed_password=hashed_password)
    print("User saved! ..not really")
    return user_in_db


@app.post("/user/", response_model=UserOut)
async def create_user(user_in: UserIn):
    user_saved = fake_save_user(user_in)
    return user_saved

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [7104]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


User saved! ..not really
INFO:     127.0.0.1:56124 - "POST /user/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7104]


## requests库

In [ ]:
url = 'http://127.0.0.1:8009/user/' 
data = {
    "username": "Foo",
    "password": "1234321",
    "email": "example@example.com"
}
res = requests.post(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"username":"Foo","email":"example@example.com","full_name":null}'

关于 **user_in.model_dump()
Pydantic 的 .model_dump()
user_in 是一个 UserIn 类的 Pydantic 模型.

Pydantic 模型具有 .model_dump（） 方法，该方法返回一个拥有模型数据的 dict。

因此，如果我们像下面这样创建一个 Pydantic 对象 user_in：

user_in = UserIn(username="john", password="secret", email="john.doe@example.com")

然后我们调用：

user_dict = user_in.model_dump()

现在我们有了一个数据位于变量 user_dict 中的 dict（它是一个 dict 而不是 Pydantic 模型对象）。

如果我们调

print(user_dict)

我们将获得一个这样的 Python dict：

{
    'username': 'john',
    'password': 'secret',
    'email': 'john.doe@example.com',
    'full_name': None,
}用：

## 解包 dict

如果我们将 user_dict 这样的 dict 以 **user_dict 形式传递给一个函数（或类），Python将对其进行「解包」。

它会将 user_dict 的键和值作为关键字参数直接传递。

因此，从上面的 user_dict 继续，编写：

UserInDB(**user_dict)

会产生类似于以下的结果：

UserInDB(
    username="john",
    password="secret",
    email="john.doe@example.com",
    full_name=Non

或者更确切地，直接使用 user_dict 来表示将来可能包含的任何内容

UserInDB(
    username = user_dict["username"],
    password = user_dict["password"],
    email = user_dict["email"],
    full_name = user_dict["full_name"],
)e,
)

## 来自于其他模型内容的 Pydantic 模型

如上例所示，我们从 user_in.model_dump（） 中获得了 user_dict，此代码：

user_dict = user_in.model_dump()
UserInDB(**user_dict

等同于：

UserInDB(**user_in.model_dump())

因为 user_in.model_dump() 是一个 dict，然后我们通过以**开头传递给 UserInDB 来使 Python「解包」它。

这样，我们获得了一个来自于其他 Pydantic 模型中的数据的 Pydantic 模型。)

## 解包 dict 和额外关键字

然后添加额外的关键字参数 hashed_password=hashed_password，例如：

UserInDB(**user_in.model_dump(), hashed_password=hashed_password)

UserInDB(
    username = user_dict["username"],
    password = user_dict["password"],
    email = user_dict["email"],
    full_name = user_dict["full_name"],
    hashed_password = hashed_password,
)

## 减少重复

减少代码重复是 FastAPI 的核心思想之一。

因为代码重复会增加出现 bug、安全性问题、代码失步问题（当你在一个位置更新了代码但没有在其他位置更新）等的可能性。

上面的这些模型都共享了大量数据，并拥有重复的属性名称和类型。

我们可以做得更好。

我们可以声明一个 UserBase 模型作为其他模型的基类。然后我们可以创建继承该模型属性（类型声明，校验等）的子类。

所有的数据转换、校验、文档生成等仍将正常运行。

这样，我们可以仅声明模型之间的差异部分（具有明文的 password、具有 hashed_password 以及不包括密码）。

In [3]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel, EmailStr

app = FastAPI()


class UserBase(BaseModel):
    username: str
    email: EmailStr
    full_name: str | None = None


class UserIn(UserBase):
    password: str


class UserOut(UserBase):
    pass


class UserInDB(UserBase):
    hashed_password: str


def fake_password_hasher(raw_password: str):
    return "supersecret" + raw_password


def fake_save_user(user_in: UserIn):
    hashed_password = fake_password_hasher(user_in.password)
    user_in_db = UserInDB(**user_in.model_dump(), hashed_password=hashed_password)
    print("User saved! ..not really")
    return user_in_db


@app.post("/user/", response_model=UserOut)
async def create_user(user_in: UserIn):
    user_saved = fake_save_user(user_in)
    return user_saved

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [7104]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


User saved! ..not really
INFO:     127.0.0.1:56149 - "POST /user/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7104]


In [ ]:
url = 'http://127.0.0.1:8009/user/' 
data = {
    "username": "Foo",
    "password": "1234321",
    "email": "example@example.com"
}
res = requests.post(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"username":"Foo","email":"example@example.com","full_name":null}'

## Union 模型

你可以将一个响应声明为两种类型的 Union，这意味着该响应将是两种类型中的任何一种。

这将在 OpenAPI 中使用 anyOf 进行定义。

为此，请使用标准的 Python 类型提示 typing.U

定义一个 Union 类型时，首先包括最详细的类型，然后是不太详细的类型。

在下面的示例中，更详细的 PlaneItem 位于 Union[PlaneItem，CarItem] 中的 CarItem 之前。nion

In [4]:
import uvicorn
from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class BaseItem(BaseModel):
    description: str
    type: str


class CarItem(BaseItem):
    type: str = "car"


class PlaneItem(BaseItem):
    type: str = "plane"
    size: int


items = {
    "item1": {"description": "All my friends drive a low rider", "type": "car"},
    "item2": {
        "description": "Music is my aeroplane, it's my aeroplane",
        "type": "plane",
        "size": 5,
    },
}


@app.get("/items/{item_id}", response_model=Union[PlaneItem, CarItem])
async def read_item(item_id: str):
    return items[item_id]

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [7104]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:56273 - "GET /items/item1 HTTP/1.1" 200 OK
INFO:     127.0.0.1:56293 - "GET /items/item2 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7104]


In [ ]:
url = 'http://127.0.0.1:8009/items/item1' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"description":"All my friends drive a low rider","type":"car"}'

In [ ]:
url = 'http://127.0.0.1:8009/items/item2' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"description":"Music is my aeroplane, it\'s my aeroplane","type":"plane","size":5}'

## 模型列表

你可以用同样的方式声明由对象列表构成的响应。

为此，请使用标准的 Python typing.List：

In [5]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str
    description: str


items = [
    {"name": "Foo", "description": "There comes my hero"},
    {"name": "Red", "description": "It's my aeroplane"},
]


@app.get("/items/", response_model=list[Item])
async def read_items():
    return items

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [7104]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:56319 - "GET /items/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7104]


In [ ]:
url = 'http://127.0.0.1:8009/items/' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 
# '[{"name":"Foo","description":"There comes my hero"},{"name":"Red","description":"It\'s my aeroplane"}]'

## 任意 dict 构成的响应

你还可以使用一个任意的普通 dict 声明响应，仅声明键和值的类型，而不使用 Pydantic 模型。

如果你事先不知道有效的字段/属性名称（对于 Pydantic 模型是必需的），这将很有用。

在这种情况下，你可以使用 typing.Dict：

In [6]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()


@app.get("/keyword-weights/", response_model=dict[str, float])
async def read_keyword_weights():
    return {"foo": 2.3, "bar": 3.4}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [7104]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:56366 - "GET /keyword-weights/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7104]


In [ ]:
url = 'http://127.0.0.1:8009/keyword-weights/' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"foo":2.3,"bar":3.4}'

## 总结

使用多个 Pydantic 模型，并针对不同场景自由地继承。

如果一个实体必须能够具有不同的「状态」，你无需为每个状态的实体定义单独的数据模

型。以用户「实体」为例，其状态有包含 password、包含 password_hash 以及不含密码。